In [2]:
import requests
import pandas as pd
import time
from datetime import datetime
from csv import writer

url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
index_data_url = "https://www.nseindia.com/api/allIndices"
headers = {
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language": "en-GB,en;q=0.9,en-US;q=0.8,hi;q=0.7,mr;q=0.6",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}
session = requests.Session()

def indices_curr_strike(url,index):
    index_data = session.get(url, headers=headers).json()['data']
    index_data_df = pd.DataFrame(index_data)
    
    df_nifty = index_data_df.loc[index_data_df['index']== index]['last']
    current_value = df_nifty.iat[0]
    int_value = current_value.astype(int)
    rounded = round(int_value/50)*50
    return rounded

def write_to_csv(expiry_date, list_data):
    with open(f"Data/N_PCR_{expiry_date}.csv", 'a', newline = '') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(list_data)
        f_object.close()

def PCR(expiry_date):
    data = session.get(url, headers=headers).json()['records']['data']
    ocdata = []
    this_moment = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    for i in data:
        for j,k in i.items():
            if j == 'CE' or j == 'PE':
                info = k
                info["instrumentType"]=j
                ocdata.append(info)

    df = pd.DataFrame(ocdata)

    #calculate expiry date Option chain data only
    c_value = indices_curr_strike(index_data_url,'NIFTY 50')

    #calculate expiry date Option chain data only
    df1 = df.loc[(df['strikePrice'] > c_value-50) & (df['strikePrice'] < c_value+100) & (df["expiryDate"] == expiry_date)]
    # print(df1.head())
    df_call = df1.loc[df1["instrumentType"] == 'CE']
    df_put = df1.loc[df1["instrumentType"] == 'PE']

    c = df_call["openInterest"].sum()
    p = df_put["openInterest"].sum()

    #Calculate PCR ratio
    PCR_rate = round(p/c,3)
    # calculate present time
    this_moment = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    if PCR_rate >= 1.05:
        todo = 'Buy'
        list_data = [this_moment,PCR_rate,todo]
        write_to_csv(expiry_date,list_data)
        print(this_moment,"-->",PCR_rate," --> 'Buy' -> Buy Call Option.")
    elif PCR_rate <= 0.90:
        todo = 'Sell'
        list_data = [this_moment,PCR_rate,todo]
        write_to_csv(expiry_date,list_data)
        print(this_moment,"-->",PCR_rate," --> 'Sell' -> Buy PUT Option.")
    else:
        print(this_moment,"-->",PCR_rate," --> Don't trade, Market is Sideways.")

    return df1

if __name__ == "__main__":
    dfame= PCR("05-Oct-2023")
    print(dfame.head(10))
    # while True:
    #     try:
    #         PCR("05-Oct-2023")
    #         time.sleep(300)
    #     except:
    #         print("connection lost")
            

2023-10-02 18:50:25 --> 0.586  --> 'Sell' -> Buy PUT Option.
     strikePrice   expiryDate underlying                       identifier  \
567        19650  05-Oct-2023      NIFTY  OPTIDXNIFTY05-10-2023CE19650.00   
568        19650  05-Oct-2023      NIFTY  OPTIDXNIFTY05-10-2023PE19650.00   
581        19700  05-Oct-2023      NIFTY  OPTIDXNIFTY05-10-2023PE19700.00   
582        19700  05-Oct-2023      NIFTY  OPTIDXNIFTY05-10-2023CE19700.00   

     openInterest  changeinOpenInterest  pchangeinOpenInterest  \
567       40679.0                -634.0              -1.534626   
568       39091.0               26406.0             208.167127   
581       63565.0               10610.0              20.035880   
582      134508.0                5317.0               4.115612   

     totalTradedVolume  impliedVolatility  lastPrice  change    pChange  \
567            2032819               7.79      88.90    3.50   4.098361   
568            1702557               8.58      76.20  -74.95 -49.586503 

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from csv import writer

url = "https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY"
index_data_url = "https://www.nseindia.com/api/allIndices"
headers = {
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language": "en-GB,en;q=0.9,en-US;q=0.8,hi;q=0.7,mr;q=0.6",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}
session = requests.Session()

def indices_curr_strike(url,index):
    index_data = session.get(url, headers=headers).json()['data']
    index_data_df = pd.DataFrame(index_data)
    
    df_nifty = index_data_df.loc[index_data_df['index']== index]['last']
    current_value = df_nifty.iat[0]
    int_value = current_value.astype(int)
    rounded = round(int_value/50)*50
    return rounded

def write_to_csv(expiry_date, list_data):
    with open(f"Data/BN_PCR_{expiry_date}.csv", 'a', newline = '') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(list_data)
        f_object.close()

def PCR(expiry_date):
    data = session.get(url, headers=headers).json()['records']['data']
    ocdata = []
    this_moment = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    for i in data:
        for j,k in i.items():
            if j == 'CE' or j == 'PE':
                info = k
                info["instrumentType"]=j
                ocdata.append(info)

    df = pd.DataFrame(ocdata)

    c_value = indices_curr_strike(index_data_url,'NIFTY BANK')

    #calculate expiry date Option chain data only
    df1 = df.loc[(df['strikePrice'] > c_value-100) & (df['strikePrice'] < c_value+200) & (df["expiryDate"] == expiry_date)]

    df_call = df1.loc[df1["instrumentType"] == 'CE']
    df_put = df1.loc[df1["instrumentType"] == 'PE']

    c = df_call["openInterest"].sum()
    p = df_put["openInterest"].sum()

    #Calculate PCR ratio
    PCR_rate = round(p/c,3)
    # calculate present time
    this_moment = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    if PCR_rate >= 1.05:
        todo = 'Buy'
        list_data = [this_moment,PCR_rate,todo]
        write_to_csv(expiry_date,list_data)
        print(this_moment,"-->",PCR_rate," --> 'Buy' -> Buy Call Option.")
    elif PCR_rate <= 0.90:
        todo = 'Sell'
        list_data = [this_moment,PCR_rate,todo]
        write_to_csv(expiry_date,list_data)
        print(this_moment,"-->",PCR_rate," --> 'Sell' -> Buy PUT Option.")
    else:
        print(this_moment,"-->",PCR_rate," --> Don't trade, Market is Sideways.")


if __name__ == "__main__":
        dfframe = PCR("04-Oct-2023")
        # while True:
        # try:
        #     PCR("04-Oct-2023")
        #     time.sleep(300)
        # except:
        #     print("connection lost")
            



In [ ]:

def write_to_csv(expiry_date, list_data):
    with open(f"BN_PCR_test{expiry_date}.csv", 'a', newline = '') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(list_data)
        f_object.close()


In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from csv import writer

index_data_url = "https://www.nseindia.com/api/allIndices"
headers = {
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language": "en-GB,en;q=0.9,en-US;q=0.8,hi;q=0.7,mr;q=0.6",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
session = requests.Session()

def indices_df(url,index):
    index_data = session.get(url, headers=headers).json()['data']
    index_data_df = pd.DataFrame(index_data)
    
    df_nifty = index_data_df.loc[index_data_df['index']== index]['last']
    current_value = df_nifty.iat[0]
    int_value = current_value.astype(int)
    rounded = round(int_value/50)*50
    return rounded

c_value = indices_df(index_data_url,'NIFTY 50')
c_value+100



In [ ]:
c_value1 = indices_df(index_data_url,'NIFTY BANK')
c_value1

In [ ]:
int_rounded.iloc[0,0]

In [ ]:
rounded = round(current_value/100)*100
int_rounded = rounded.astype(int)
int_rounded

In [ ]:
def indices_df(url):
    index_data = session.get(url, headers=headers).json()['data']
    index_data_df = pd.DataFrame(index_data)
    return index_data_df

df222 = indices_df(index_data_url)